In [3]:
from pathlib import Path
from typing import List

import pandas as pd
import numpy as np

YEAR = 2022


A line magic that can help to download the data and store it in variables
`input_path` and `data` to use directly in the code after:

It will use:
* if no argument: current day and year
* one argument: the day set as argument + current year
* two arguments: the day and the year

```python
%get_daily_input 2
data[:100]
```

In [153]:
!pip install -q advent-of-code-data

You should consider upgrading via the 'C:\AnsysDev\Python39\python.exe -m pip install --upgrade pip' command.


In [155]:
from pathlib import Path
from typing import cast
from datetime import datetime

from aocd import get_data

from IPython import get_ipython  # type: ignore
from IPython.core.interactiveshell import InteractiveShell  # type: ignore
from IPython.core.magic import register_line_cell_magic  # type: ignore
from IPython.display import HTML  # type: ignore


@register_line_cell_magic
def get_daily_input(line:str) -> None:
    """Retrieve the input for the current day and storing variables
    * input_path: Path to the input file
    * data: Content of the input file
    """
    ipython = cast(InteractiveShell, get_ipython())

    today = datetime.today()
    current_year = today.year
    current_day = today.day
    if line:
        args = [int(e) for e in line.split()] + [current_year]
    else:
        args = [current_day, current_year]
    day, year = args[:2]

    input_dir = Path("inputs")
    input_dir.mkdir(exist_ok=True)
    input_path = input_dir / f"{day:02d}.txt"
    input_path_html = f'<a href="{input_path.as_posix()}">{input_path.as_posix()}</a>'

    if not input_path.exists():
        ipython.system(f"aocd {day} {year} > {input_path}")
        ipython.run_cell(f"HTML('✅Input saved to {input_path_html}')")
    else:
        ipython.run_cell(f"HTML('✔️Input already exists in {input_path_html}')")

    ipython.run_cell(f"input_path = Path('{input_path.as_posix()}')")
    ipython.run_cell(f"data = get_data(day={day}, year={year})")
    msg = "✅Variables <code>input_path</code> and <code>data</code> set"
    ipython.run_cell(f"HTML('{msg}')")

# 01 December 2022


In [156]:
%get_daily_input 1 2022

In [157]:
def get_calories(path: Path) -> List[int]:
    """Retrieve calories carried by each elf"""
    content = Path(path).read_text(encoding="utf-8")
    return [
        sum(int(line) for line in block.strip().split("\n"))
        for block in content.split("\n\n")
    ]

calories = pd.Series(get_calories(input_path), name="calories")
calories


0      55684
1      43399
2      48846
3      48058
4      52061
       ...  
245    22318
246    48674
247    43489
248    47659
249    50009
Name: calories, Length: 250, dtype: int64

In [111]:
# Get the top calories
calories.max()


69693

In [12]:
# Get the sum of calories from top 3
calories.sort_values(ascending=False).head(3).sum()


200945

# 02 December 2022


In [160]:
%get_daily_input 2
data[:100]

'A Z\nA Y\nB X\nB X\nC X\nB X\nA X\nA X\nC X\nA X\nA X\nA Y\nB X\nA Y\nC X\nC X\nA X\nA Y\nC X\nB X\nA X\nB X\nA X\nA X\nB Y\n'

In [176]:
import enum


class Gesture(enum.IntEnum):
    ROCK = 0
    PAPER = 1
    SCISSORS = 2

GESTURES = list(Gesture)

In [177]:
from typing import Literal

ABC = Literal["A", "B", "C"]
XYZ = Literal["X", "Y", "Z"]


def get_points(other_letter: ABC, my_letter: XYZ) -> int:
    """Get the points for a given combination of letters"""
    # We will use the values of the enum
    other = GESTURES["ABC".index(other_letter)]
    me = GESTURES["XYZ".index(my_letter)]

    points_choice = me.value + 1
    if other == me:
        return 3 + points_choice

    if me.value == (other.value + 1) % 3:
        return 6 + points_choice

    return points_choice


get_points("A", "Y"), get_points("B", "X"), get_points("C", "Z")


(8, 1, 6)

In [178]:
mini_data = """
A Y
B X
C Z
"""

In [179]:
import re


def get_points_from_list(data: str, fn=get_points) -> int:
    """Get the points from the data"""
    return sum(
        fn(other, my) for other, my in re.findall(r"([ABC]) ([XYZ])", data)
    )


(get_points_from_list(mini_data), get_points_from_list(data))


(15, 11150)

In [181]:
def get_points_2(other_letter: ABC, my_letter: XYZ) -> int:
    """Get the points for a given combination of letters (2nd scenario)"""
    other = GESTURES["ABC".index(other_letter)]

    # X means you loose, Y means you draw, Z means you win
    offset = "XYZ".index(my_letter) - 1
    me = GESTURES[(other.value + offset) % 3]

    points_choice = me.value + 1
    if other == me:
        return 3 + points_choice

    if me.value == (other.value + 1) % 3:
        return 6 + points_choice

    return points_choice


assert get_points_from_list(mini_data, fn=get_points_2) == 6 + 3 + 3
get_points_from_list(data, fn=get_points_2)

8295

# Day 3

In [139]:
%get_daily_input 3
data[:100]

'BdbzzddChsWrRFbzBrszbhWMLNJHLLLLHZtSLglFNZHLJH\nnnfMwqpQTMffHlNNLllHnZSS\ncGpcMwfppfqcjcTCBBzWDsDbDrjz'

In [142]:
MINI_DATA = """\
vJrwpWtwJgWrhcsFMMfFFhFp
jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL
PmmdzqPrVvPwwTWBwg
wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn
ttgJtRGJQctTZtZT
CrZsJsPPZsGzwwsLwLmpwMDw
"""

In [150]:
from string import ascii_lowercase, ascii_uppercase

def get_line_priority(line: str) -> int:
    """Get the priority of each rucksack line"""
    half_length = len(line) // 2
    left, right = line[:half_length], line[half_length:]
    common_letter = set(left).intersection(set(right)).pop()
    # Find the priority of the common letter
    letters = ascii_lowercase + ascii_uppercase
    return letters.index(common_letter) + 1

get_line_priority("vJrwpWtwJgWrhcsFMMfFFhFp"), get_line_priority("jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL")

(16, 38)

In [151]:
sum(get_line_priority(line) for line in data.splitlines())

7889

In [148]:
from typing import Generator


def get_badge_priority(txt: str) -> Generator[int, None, None]:
    """Get the priorities of the badges, by group of 3"""
    lines = txt.splitlines()
    for i in range(0, len(lines), 3):
        common_letters = (
            set(lines[i])
            .intersection(set(lines[i + 1]))
            .intersection(set(lines[i + 2]))
        )
        letters = ascii_lowercase + ascii_uppercase
        yield min(letters.index(letter) + 1 for letter in common_letters)


sum(get_badge_priority(MINI_DATA))


70

In [149]:
sum(get_badge_priority(data))

2825

# Day 4

In [128]:
%get_daily_input 4
data[:100]

'91-93,6-92\n85-97,18-63\n15-99,16-98\n35-49,34-50\n4-7,6-93\n57-86,57-58\n1-82,83-90\n21-88,20-89\n83-84,1-8'

In [129]:
MINI_DATA = """\
2-4,6-8
2-3,4-5
5-7,7-9
2-8,3-7
6-6,4-6
2-6,4-8"""

In [131]:
def is_fully_contained(line:str) -> bool:
    """Check if the line is fully contained in another line"""
    left, right = line.split(",")
    left_min, left_max = [int(e) for e in left.split("-")]
    right_min, right_max = [int(e) for e in right.split("-")]
    return (left_min <= right_min and left_max >= right_max) or (right_min <= left_min and right_max >= left_max)

is_fully_contained("6-6,4-6"), is_fully_contained("2-6,4-8")

(True, False)

In [132]:
sum(is_fully_contained(line) for line in MINI_DATA.splitlines())

2

In [133]:
sum(is_fully_contained(line) for line in data.splitlines())

456

In [135]:
def is_overlap(line:str) -> bool:
    """Check if the line contains overlap"""
    left, right = line.split(",")
    left_min, left_max = [int(e) for e in left.split("-")]
    right_min, right_max = [int(e) for e in right.split("-")]
    left_range = set(range(left_min, left_max + 1))
    right_range = set(range(right_min, right_max + 1))
    return bool(left_range.intersection(right_range))

is_overlap("6-6,4-6"), is_overlap("2-6,4-8"), is_overlap("4-8,2-6"), is_overlap("2-4,6-8")

(True, True, True, False)

In [136]:
sum(is_overlap(line) for line in data.splitlines())

808